In [1]:
from keras.models import Model
from keras.layers import Flatten,Dense
from keras.applications.vgg16 import VGG16 #Import all the necessary modules
import matplotlib.pyplot as plot
from glob import glob

In [2]:
IMAGESHAPE = [224, 224, 3] #Provide image size as 224 x 224 this is a fixed-size for VGG16 architecture
vgg_model = VGG16(input_shape=IMAGESHAPE, weights='imagenet', include_top=False)
#3 signifies that we are working with RGB type of images.
training_data = 'chest_xray/train'
testing_data = 'chest_xray/test'

In [3]:

for each_layer in vgg_model.layers:
	each_layer.trainable = False #Set the trainable as False, So that all the layers would not be trained.
classes = glob('chest_xray/train/*') #Finding how many classes present in our train dataset.
flatten_layer = Flatten()(vgg_model.output)
prediction = Dense(len(classes), activation='softmax')(flatten_layer)
final_model = Model(inputs=vgg_model.input, outputs=prediction) #Combine the VGG output and prediction , this all together will create a model.
final_model.summary() #Displaying the summary
final_model.compile( #Compiling our model using adam optimizer and optimization metric as accuracy.
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [4]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, #importing our dataset to keras using ImageDataGenerator in keras.
								shear_range = 0.2,
								zoom_range = 0.2,
								horizontal_flip = True)
testing_datagen = ImageDataGenerator(rescale =1. / 255)
training_set = train_datagen.flow_from_directory('chest_xray/train', #inserting the images.
												target_size = (224, 224),
												batch_size = 4,
												class_mode = 'categorical')
test_set = testing_datagen.flow_from_directory('chest_xray/test',
											target_size = (224, 224),
											batch_size = 4,
											class_mode = 'categorical')
fitted_model = final_model.fit_generator( #Fitting the model.
training_set,
validation_data=test_set,
epochs=5,
steps_per_epoch=len(training_set),
validation_steps=len(test_set)
)
print(final_model)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


C:\Users\HP\AppData\Local\Temp\ipykernel_2628\1229750149.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fitted_model = final_model.fit_generator( #Fitting the model.


Epoch 1/5
1304/1304 [==============================] - 2142s 2s/step - loss: 0.2996 - accuracy: 0.9270 - val_loss: 1.4934 - val_accuracy: 0.7724
Epoch 2/5
1304/1304 [==============================] - 1837s 1s/step - loss: 0.2551 - accuracy: 0.9465 - val_loss: 1.3162 - val_accuracy: 0.8462
Epoch 3/5
1304/1304 [==============================] - 1954s 1s/step - loss: 0.2148 - accuracy: 0.9544 - val_loss: 0.4603 - val_accuracy: 0.9263
Epoch 4/5
1304/1304 [==============================] - 1981s 2s/step - loss: 0.2392 - accuracy: 0.9540 - val_loss: 0.8069 - val_accuracy: 0.8926
Epoch 5/5
1304/1304 [==============================] - 1923s 1s/step - loss: 0.2152 - accuracy: 0.9578 - val_loss: 0.9252 - val_accuracy: 0.9054


In [ ]:
final_model.save('model.h5')

In [ ]:
x=open("my_model.h5","w")
x.write(final_model)
x.close()

In [ ]:
plot.plot(fitted_model.history['loss'], label='training loss') #Plotting the accuracies
plot.plot(fitted_model.history['val_loss'], label='validation loss')
plot.legend()
plot.show()
plot.savefig('LossVal_loss')
plot.plot(fitted_model.history['acc'], label='training accuracy')
plot.plot(fitted_model.history['val_acc'], label='validation accuracy')
plot.legend()
plot.show()
plot.savefig('AccVal_acc')
 #Saving the model file.
